In [14]:
import random

import pandas as pd
import numpy as np
from scipy import stats

In [15]:
data = pd.read_csv("../data/csv/achmusik.csv")

data = data[data.columns[:-2]]
# Getting the decimals right -- commas to points and no more Nones
data = data.set_index(["genre", "sub_genre", "artist", "album", "song"])
data.fillna(value="", inplace=True)

for i in range(data.columns.size):
    data[data.columns[i]] = pd.to_numeric(data[data.columns[i]], errors='coerce')

data

Qu  \
genre   sub_genre   artist             album                         song                           
Reggae  NaN         10cc               Bloody Tourists               Dreadlock Holiday       7.25   
Rock    Soft Rock   10cc               The Original Soundtrack       I'm Not In Love         4.50   
Hip-Hop Gangsta     2Pac               All Eyez On Me                All Eyez On Me          8.00   
                                                                     Ambitionz Az A Ridah    6.75   
Rock    Alternative 4 Non Blondes      Bigger, Better, Faster, More! What's Up               8.75   
...                                                                                           ...   
Disco   Funk        Earth, Wind & Fire I Am                          Can't Let Go            8.00   
                                                                     Let Your Feelings Show  7.50   
                                                                     Star                    8.00   
                                                                     Rock That!              7.25   
                                                                     In the Stone            8.25   

                                                                                               Gr  \
genre   sub_genre   artist             album                         song                           
Reggae  NaN         10cc               Bloody Tourists               Dreadlock Holiday       7.00   
Rock    Soft Rock   10cc               The Original Soundtrack       I'm Not In Love         7.60   
Hip-Hop Gangsta     2Pac               All Eyez On Me                All Eyez On Me          7.00   
                                                                     Ambitionz Az A Ridah    7.75   
Rock    Alternative 4 Non Blondes      Bigger, Better, Faster, More! What's Up               8.50   
...                                                                                           ...   
Disco   Funk        Earth, Wind & Fire I Am                          Can't Let Go             NaN   
                                                                     Let Your Feelings Show   NaN   
                                                                     Star                     NaN   
                                                                     Rock That!               NaN   
                                                                     In the Stone             NaN   

                                                                                              Vi  \
genre   sub_genre   artist             album                         song                          
Reggae  NaN         10cc               Bloody Tourists               Dreadlock Holiday       7.0   
Rock    Soft Rock   10cc               The Original Soundtrack       I'm Not In Love         6.0   
Hip-Hop Gangsta     2Pac               All Eyez On Me                All Eyez On Me          7.0   
                                                                     Ambitionz Az A Ridah    7.0   
Rock    Alternative 4 Non Blondes      Bigger, Better, Faster, More! What's Up               7.0   
...                                                                                          ...   
Disco   Funk        Earth, Wind & Fire I Am                          Can't Let Go            NaN   
                                                                     Let Your Feelings Show  NaN   
                                                                     Star                    NaN   
                                                                     Rock That!              NaN   
                                                                     In the Stone            NaN   

                                                                                              Ro  \
genre   sub_genre   artist             album                      

In [16]:
qu_df = pd.DataFrame(columns=["mean", "ok"])
qu_df["ok"] = data["Qu"].dropna().groupby("artist").size() > 4
qu_df["mean"] = data["Qu"].dropna().groupby("artist").mean()

qu_df = qu_df[qu_df["ok"] == True].sort_values(by="mean", ascending=False)
qu_df

,mean,ok
artist,,
Georges Brassens,8.458333,True
Ultra Vomit,8.400000,True
Supertramp,8.392857,True
Golden Earring,8.375000,True
Iron Maiden,8.375000,True
...,...,...
Dr. Dre,5.050000,True
Caballero,4.840909,True
Neurosis,4.166667,True


In [44]:
data.merge(data["Qu"].groupby("artist").size() > 5, on="artist")
# data[]["Qu"].groupby("artist").mean()

,Qu_x,Gr,Vi,Ro,Sa,Gl,Rx,Cl,Lu,Gë,Et,Ti,Qu_y
artist,,,,,,,,,,,,,
10cc,7.25,7.00,7.00,NaN,NaN,NaN,NaN,NaN,6.80,NaN,NaN,NaN,False
10cc,4.50,7.60,6.00,NaN,NaN,NaN,NaN,NaN,6.50,NaN,NaN,NaN,False
2Pac,8.00,7.00,7.00,NaN,NaN,NaN,NaN,NaN,7.75,NaN,NaN,NaN,False
2Pac,6.75,7.75,7.00,NaN,NaN,NaN,NaN,NaN,7.75,NaN,NaN,NaN,False
4 Non Blondes,8.75,8.50,7.00,8.7,NaN,NaN,NaN,7.1,8.88,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Brian Bennett,7.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,False
Brothers of Metal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
Theophany,6.50,NaN,6.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.5,False
